In [52]:
import numpy as np
import scipy as sp
import pandas as pd

import re

from IPython.display import display

# Aula 03 &mdash; Introdução a Pandas - Parte 2

Renato Vimieiro
rv2 {em} cin.ufpe.br

março 2017

# Resumo

- Nessa aula continuaremos a explorar Pandas
- Continuaremos a usar os dados do MovieLens como exemplo
- Reutilizaremos os dados descritos na aula 02

In [2]:
usuarios = pd.read_csv(
    "http://files.grouplens.org/datasets/movielens/ml-100k/u.user",
    sep='|',header=None, names=["user_id", "age", "gender", "occupation", "zip_code"])
filmes = pd.read_csv(
    "http://files.grouplens.org/datasets/movielens/ml-100k/u.item",
    sep='|',header=None, names=["movie_id", "movie_title",  "release_date", "video_release_date", "IMDb_URL", "unknown", "Action", "Adventure", "Animation", 
        "Children", "Comedy", "Crime", "Documentary", "Drama", "Fantasy","FilmNoir", "Horror", "Musical", "Mystery", "Romance", "SciFi","Thriller", "War", "Western"])
avaliacoes = pd.read_csv(
    "http://files.grouplens.org/datasets/movielens/ml-100k/u.data",
    sep='\t',header=None, names=["user_id", "movie_id", "rating", "timestamp"])


# Indexação e seleção de dados

In [3]:
# Selecionando uma coluna
print(usuarios['age'].head())
print(usuarios.age.head())

0    24
1    53
2    23
3    24
4    33
Name: age, dtype: int64
0    24
1    53
2    23
3    24
4    33
Name: age, dtype: int64


In [5]:
# Selecionando multiplas colunas

usuarios[['age','gender']].head()

,age,gender
0,24,M
1,53,F
2,23,M
3,24,M
4,33,F


In [11]:
# Selecionando linhas com slice
avaliacoes[10:15]

,user_id,movie_id,rating,timestamp
10,62,257,2,879372434
11,286,1014,5,879781125
12,200,222,5,876042340
13,210,40,3,891035994
14,224,29,3,888104457


In [13]:
# Nao e possivel combinar selecao de linha e coluna diretamente via []
avaliacoes[1:5,'rating']

TypeError: unhashable type: 'slice'

In [85]:
# A selecao de porcoes especificas sao feitas atraves dos atributos
# loc permite selecao por labels
print(avaliacoes.loc[10:15,'rating'])

# iloc permite a selecao por indice
display(filmes.iloc[:5,[1,2,4]])

10    2
11    5
12    5
13    3
14    3
15    3
Name: rating, dtype: int64


,movie_title,release_date,IMDb_URL
0,Toy Story (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Toy%20Story%2...
1,GoldenEye (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?GoldenEye%20(...
2,Four Rooms (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Four%20Rooms%...
3,Get Shorty (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Get%20Shorty%...
4,Copycat (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Copycat%20(1995)


In [87]:
# Ainda podemos selecionar os dados com mascaras 
# booleanas como em NumPy
usuarios[(usuarios.age > 40) & 
         ~(usuarios.occupation.isin(['none','other']))].head()

,user_id,age,gender,occupation,zip_code
5,6,42,M,executive,98101
6,7,57,M,administrator,91344
9,10,53,M,lawyer,90703
12,13,47,M,educator,29206
13,14,45,M,scientist,55106


Exercício:
======

- Mostre a média de idade das mulheres cientistas
- Quantos filmes de animação foram lançados em 1968

In [89]:
cientistas = usuarios[(usuarios.gender=='F') 
                      & (usuarios.occupation=='scientist')]
print(cientistas.head())
print(cientistas.age.mean())

     user_id  age gender occupation zip_code
174      175   26      F  scientist    21911
729      730   31      F  scientist    32114
929      930   28      F  scientist    07310
28.3333333333


In [91]:
print(filmes[(filmes.Animation==1)&
             filmes.release_date.str.contains('1968')].shape[0])

1


# Atribuições de valores

- A indexação através de `loc` e `iloc` retorna uma visão do data frame
- Essa visão pode ser modificada como fizemos em NumPy
- Além disso, podemos também acrescentar novos dados

In [93]:
c2 = cientistas.copy()
c2.iloc[0,1] = -1
c2.loc[:,'zip_code'] = None
c2.head()

,user_id,age,gender,occupation,zip_code
174,175,-1,F,scientist,None
729,730,31,F,scientist,None
929,930,28,F,scientist,None


In [95]:
filmes['release_year'] = filmes.release_date.apply(
        lambda x: not x is np.nan and 
    re.search("\d+\-\w+\-(\d+)",str(x)).group(1) or None)
filmes[['movie_title','release_date','release_year']].head()

,movie_title,release_date,release_year
0,Toy Story (1995),01-Jan-1995,1995
1,GoldenEye (1995),01-Jan-1995,1995
2,Four Rooms (1995),01-Jan-1995,1995
3,Get Shorty (1995),01-Jan-1995,1995
4,Copycat (1995),01-Jan-1995,1995


# Operações básicas

- Pandas conta com várias funções pré-definidas para obter estatísticas e informações básicas dos dados
- Ela conta também, como vimos anteriormente, com uma função `apply` que aplica uma função aos elementos
- Seguem alguns exemplos

In [105]:
print("Mediana de idade dos usuarios ", usuarios.age.median())

piorAvaliacao = avaliacoes.rating.argmin()
print(filmes[filmes.movie_id == avaliacoes.movie_id.iloc[piorAvaliacao]].movie_title.iloc[0])

avaliacoes.rating.value_counts()

Mediana de idade dos usuarios  31.0
Heavyweights (1994)


4    34174
3    27145
5    21201
2    11370
1     6110
Name: rating, dtype: int64

# Concat, merge e joins

- Vimos no exemplo anterior que pode ser necessário fazer cruzamento de dados em diferentes tabelas
- Essa é uma situação muito comum em bancos de dados, onde tabelas são frequentemente cruzadas
- Pandas dispõe de mecanismos similares para cruzamento de dados

In [119]:
# A forma mais simples de juntar diferentes dados e concatenacao
A = pd.Series(["A{}".format(a) for a in range(4)],index=range(4), name="A")
B = pd.Series(["B{}".format(a) for a in range(5)],index=range(5), name= "B")
C = pd.Series(["C{}".format(a) for a in range(5)],index=range(5), name= "C")
pd.concat([A,B,C],axis=1)

,A,B,C
0,A0,B0,C0
1,A1,B1,C1
2,A2,B2,C2
3,A3,B3,C3
4,NaN,B4,C4


# Merge 

- Uma opção mais [SQL-like](https://www.codeproject.com/KB/database/Visual_SQL_Joins/Visual_SQL_JOINS_orig.jpg) é merge
- De acordo com a documentação essa função foi otimizada para desempenho
- A sintaxe é

```python
pd.merge(left, right, how='inner', 
on=None, left_on=None, right_on=None,
left_index=False, right_index=False, sort=True,
suffixes=('_x', '_y'),copy=True,indicator=False)
```         

In [120]:
avaliacaoFilmes = pd.merge(avaliacoes,filmes[["movie_id","movie_title","release_year"]], on="movie_id")
avaliacaoFilmes.head()

,user_id,movie_id,rating,timestamp,movie_title,release_year
0,196,242,3,881250949,Kolya (1996),1997
1,63,242,3,875747190,Kolya (1996),1997
2,226,242,5,883888671,Kolya (1996),1997
3,154,242,3,879138235,Kolya (1996),1997
4,306,242,5,876503793,Kolya (1996),1997


Exercício:
======

- Calcule a média e desvio padrão das avaliações dadas aos filmes da década de 80 por programadores

In [140]:
avaliacaoFilmes.release_year.fillna(value=-1,inplace=True)
avaliacaoFilmes['release_year'] = avaliacaoFilmes.release_year.astype(np.int)
avalFilmUsu = pd.merge(avaliacaoFilmes,usuarios[["user_id","occupation"]], on="user_id")
selecao = avalFilmUsu[(1990 <= avalFilmUsu.release_year) & 
            (avalFilmUsu['release_year'] < 2000) 
            & (avalFilmUsu.occupation == 'programmer')]
selecao.rating.mean(), selecao.rating.std() 


(3.4322604790419162, 1.1414533040854842)